In [1]:
#this work evaluate and defent the malware classifer of Grosses et al. 2016

In [2]:
import numpy as np
import matplotlib.pylab as plt

import os
import sys
import csv
import collections
import pickle
import re

In [3]:
manifest = ['activity', 'permission', 'intent', 'feature', 'service_receiver']

def _filter_token(token):
    feats = token.split("::")
    if feats[0] in manifest:
        return token
    else:
        return
    

In [4]:
#Drebin dataset directory
Dre_dir = './dataset/'
#Load features vector 
features_dataset_dir = os.path.join(Dre_dir, 'feature_vectors')
features_vectors_files = os.listdir(features_dataset_dir)
#save features vector to list structure
feature_vectors = []
for i, e in enumerate(features_vectors_files):
    file_path = os.path.join(features_dataset_dir, e)
    f = open(file_path, 'r')
    feat_vect = []
    for l in f:
        #Only preserve the features in Manifest.XML file
        l_perserved = _filter_token(l.strip())
        
        if len(l.strip()) and l_perserved != None:
            feat_vect.append(l.strip())
    feature_vectors.append(feat_vect)
    f.close()

for i in range(5):
    print("[%d] features vector is [%s]" % (i, feature_vectors[i][:5]))

[0] features vector is [['feature::android.hardware.touchscreen', 'permission::android.permission.ACCESS_FINE_LOCATION', 'activity::.LauncherSplashActivity', 'permission::android.permission.INTERNET', 'service_receiver::com.atlatl.cda.android.app.service.FlatFormDloadIntentService']]
[1] features vector is [['feature::android.hardware.touchscreen', 'permission::android.permission.INTERNET', 'activity::.SUL', 'intent::android.intent.action.MAIN', 'permission::android.permission.ACCESS_NETWORK_STATE']]
[2] features vector is [['feature::android.hardware.touchscreen', 'permission::android.permission.ACCESS_FINE_LOCATION', 'permission::android.permission.INTERNET', 'intent::android.intent.category.DEFAULT', 'feature::android.hardware.location.gps']]
[3] features vector is [['feature::android.hardware.touchscreen', 'intent::android.intent.action.MAIN', 'permission::android.permission.INTERNET', 'intent::android.intent.category.LAUNCHER', 'activity::.CalculatorHouseLoan']]
[4] features vecto

# 1.COUNT WORDS

In [9]:
c = collections.Counter()
for feat_vector in feature_vectors:
    for feat in feat_vector:
        c[feat] = c[feat] + 1

print ('The counter item is [%s], the counter length is [%d]' % \
      (type(c.items()), len(c.items())))
for i in range(5):
    print("%d the item is [%s]" % (i, c.items()[i]))

The counter item is [<type 'list'>], the counter length is [229214]
0 the item is [('activity::.ScanActivity1', 1)]
1 the item is [('activity::com.doapps.android.realestate.listingdetails.OAuthTwitter', 6)]
2 the item is [('activity::.ResListActivity', 1)]
3 the item is [('service_receiver::.Y5Service', 1)]
4 the item is [('activity::.Acknowledgements', 3)]


# 2.Sort the Count 

In [10]:
LOW_FREQS = 6
for c_i in list(c):
    if c[c_i] < LOW_FREQS:
        del c[c_i]
c_pairs = sorted(c.items(), key = lambda x: -x[1])

print ('After delete low frequency token, the counter item is [%s], the counter length is [%d]' % \
      (type(c.items()), len(c.items())))

After delete low frequency token, the counter item is [<type 'list'>], the counter length is [13641]


# 3.Make Dictionary 

In [11]:
words, counts = zip(*(c_pairs))
vocab = dict(zip(words, range(len(words))))
print("Type of 'words' is [%s], the length of 'words' is [%d]" %\
     (type(words), len(words)))
print("Type of 'counts' is [%s], the length of 'counts' is [%d]" %\
     (type(counts), len(counts)))
print("Type of 'vocab' is [%s], the length of 'vocab' is [%d]" % \
     (type(vocab), len(vocab)))

Type of 'words' is [<type 'tuple'>], the length of 'words' is [13641]
Type of 'counts' is [<type 'tuple'>], the length of 'counts' is [13641]
Type of 'vocab' is [<type 'dict'>], the length of 'vocab' is [13641]


# 4. The Usage of 'words' & 'vocab'

In [12]:
#number to words
print('=========words usage=========')
for i in range(5):
    print(" [%d]: words[%d] is [%s]" % (i, i, words[i]))
print('=========vocab usage=========')
for i in range(5):
    print(" [%d]: [vocab[%s]] is [%s]" % (i, words[i], vocab[words[i]]))

=========words usage=========
 [0]: words[0] is [feature::android.hardware.touchscreen]
 [1]: words[1] is [intent::android.intent.action.MAIN]
 [2]: words[2] is [intent::android.intent.category.LAUNCHER]
 [3]: words[3] is [permission::android.permission.INTERNET]
 [4]: words[4] is [permission::android.permission.ACCESS_NETWORK_STATE]
=========vocab usage=========
 [0]: [vocab[feature::android.hardware.touchscreen]] is [0]
 [1]: [vocab[intent::android.intent.action.MAIN]] is [1]
 [2]: [vocab[intent::android.intent.category.LAUNCHER]] is [2]
 [3]: [vocab[permission::android.permission.INTERNET]] is [3]
 [4]: [vocab[permission::android.permission.ACCESS_NETWORK_STATE]] is [4]


In [13]:
#select training and testing dataset
malware_sha256_path = os.path.join(Dre_dir, 'sha256_family.csv')
fcsv = open(malware_sha256_path, 'rb')
reader = csv.reader(fcsv)
malware_sha256 = []
for line in reader:
    if len(line[0].strip()) >= 10:
        malware_sha256.append(line[0].strip())
print malware_sha256[:5]

['090b5be26bcc4df6186124c2b47831eb96761fcf61282d63e13fa235a20c7539', 'bedf51a5732d94c173bcd8ed918333954f5a78307c2a2f064b97b43278330f54', '149bde78b32be3c4c25379dd6c3310ce08eaf58804067a9870cfe7b4f51e62fe', 'dd11c105ec8bb3c851f5955fa53eebb91b7dc46bef4d919ee4b099e825c56325', '6832234c4eae7a57be4f68271b7eecb056c4cd8352c67d2273d676208118871d']


In [14]:
malware_feature_vectors = []
malware_idx = []
for _sha256 in malware_sha256:
    idx = features_vectors_files.index(_sha256)
    if idx > -1:
        malware_feature_vectors.append(feature_vectors[idx])
        malware_idx.append(idx)


benign_feature_vectors = []
for _i, _e in enumerate(feature_vectors):
    if _i not in malware_idx:
        benign_feature_vectors.append(_e)

print("The length of malware is %d : %d is for the length of benign" % \
     (len(malware_feature_vectors), len(benign_feature_vectors)))  

The length of malware is 5560 : 123453 is for the length of benign


# 6.Extract features 

In [15]:
input_dims = len(vocab)
malware_binary_data = []
for _vector in malware_feature_vectors:
    data = np.zeros((input_dims),dtype = np.float32)
    if len(_vector) > 0 :
        value_idx = [v for v in list(map(vocab.get, _vector)) if v is not None]
        data[np.array(value_idx)] = 1.
    malware_binary_data.append(data)
malware_labels = [1] * len(malware_binary_data)

benign_binary_data = []
for _vector in benign_feature_vectors:
    data = np.zeros((input_dims),dtype = np.float32)
    if len(_vector) > 0:
        value_idx = [v for v in list(map(vocab.get, _vector)) if v is not None]
        data[np.array(value_idx)] = 1.
    benign_binary_data.append(data)
benign_labels = [0] * len(benign_binary_data)  

# 5.Save & Load 

In [16]:
import cPickle
save_dir = Dre_dir
save_name = os.path.join(save_dir, 'words_vocab.pkl')
with open(save_name, 'w') as fsave:
    cPickle.dump((words, vocab), fsave)
    print("Sequence and vocab is saved to [%s]" % save_name)


#load 
with open(save_name, 'rb') as fload:
    words2, vocab2 = pickle.load(fload)
    print('=========words usage=========')
    for i in range(5):
        print(" [%d]: words[%d] is [%s]" % (i, i, words2[i]))
    print('=========vocab usage=========')
    for i in range(5):
        print(" [%d]: [vocab[%s]] is [%s]" % (i, words2[i], vocab2[words2[i]]))

Sequence and vocab is saved to [./dataset/words_vocab.pkl]
=========words usage=========
 [0]: words[0] is [feature::android.hardware.touchscreen]
 [1]: words[1] is [intent::android.intent.action.MAIN]
 [2]: words[2] is [intent::android.intent.category.LAUNCHER]
 [3]: words[3] is [permission::android.permission.INTERNET]
 [4]: words[4] is [permission::android.permission.ACCESS_NETWORK_STATE]
=========vocab usage=========
 [0]: [vocab[feature::android.hardware.touchscreen]] is [0]
 [1]: [vocab[intent::android.intent.action.MAIN]] is [1]
 [2]: [vocab[intent::android.intent.category.LAUNCHER]] is [2]
 [3]: [vocab[permission::android.permission.INTERNET]] is [3]
 [4]: [vocab[permission::android.permission.ACCESS_NETWORK_STATE]] is [4]


In [19]:
#save data 
mal_data_path = os.path.join(save_dir, 'malware.data')
with open(mal_data_path, 'wb') as fsave:
    np.save(fsave, np.array(malware_binary_data, dtype= np.float32))
ben_data_path = os.path.join(save_dir, 'benign.data')    
with open(ben_data_path, 'wb') as fsave:
    np.save(fsave, np.array(benign_binary_data, dtype= np.float32))

with open(mal_data_path, 'rb') as f_r:
    mal_data = np.load(f_r)
print(mal_data.shape)    

(5560, 13641)
